In [1]:
# 거리 값들을 사용해서 선형 회귀 모델
# 스파크 세션 생성
from pyspark.sql import SparkSession

# 인스턴스 생성(Max Memory 지정: Out of Memory 방지)
MAX_MEMORY="10g"
spark = SparkSession.builder.appName("taxi-fare-prediciton")\
                .config("spark.executor.memory", MAX_MEMORY)\
                .config("spark.driver.memory", MAX_MEMORY)\
                .master("local[*]")\
                .getOrCreate()

# 인스턴스 생성 시 부터 맥스 메모리(Max Memory)를 지정해준다.
# Out of Memory 증상을 미연에 방지한다.


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/05/16 22:12:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/05/16 22:12:08 WARN Utils: Service 'sparkDriver' could not bind on a random free port. You may check whether configuring an appropriate binding address.
23/05/16 22:12:08 WARN Utils: Service 'sparkDriver' could not bind on a random free port. You may check whether configuring an appropriate binding address.
23/05/16 22:12:08 WARN Utils: Service 'sparkDriver' could not bind on a random free port. You may check whether configuring an appropriate binding address.
23/05/16 22:12:08 WARN Utils: Service 'sparkDriver' could not bind on a random free port. You may check whether configuring an appropriate binding address.
23/05/16 22:12:08 WARN Utils: Service 'sparkDriver' could not bind on a random free port. You may check whether configuring an appropriate binding address.
23/05/16 22:12:08 WARN Utils: Service 'sparkDriver' could not bind on a rando

Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: java.net.BindException: Can't assign requested address: Service 'sparkDriver' failed after 16 retries (on a random free port)! Consider explicitly setting the appropriate binding address for the service 'sparkDriver' (for example spark.driver.bindAddress for SparkDriver) to the correct binding address.
	at java.base/sun.nio.ch.Net.bind0(Native Method)
	at java.base/sun.nio.ch.Net.bind(Net.java:479)
	at java.base/sun.nio.ch.Net.bind(Net.java:468)
	at java.base/sun.nio.ch.ServerSocketChannelImpl.bind(ServerSocketChannelImpl.java:220)
	at io.netty.channel.socket.nio.NioServerSocketChannel.doBind(NioServerSocketChannel.java:134)
	at io.netty.channel.AbstractChannel$AbstractUnsafe.bind(AbstractChannel.java:562)
	at io.netty.channel.DefaultChannelPipeline$HeadContext.bind(DefaultChannelPipeline.java:1334)
	at io.netty.channel.AbstractChannelHandlerContext.invokeBind(AbstractChannelHandlerContext

In [2]:
# 파일 불러오기
# trips_df = spark.read.csv("./data/trips/*", inferSchema=True, header=True)
trips_df = spark.read.parquet("./data/trips/parquet/*", inferSchema=True, header=True)

# 스키마 확인
trips_df.printSchema()
'''
-csv
root
 |-- _c0: string (nullable = true)
 |-- VendorID: string (nullable = true)
 |-- tpep_pickup_datetime: string (nullable = true)
 |-- tpep_dropoff_datetime: string (nullable = true)
 |-- passenger_count: string (nullable = true)
 |-- trip_distance: string (nullable = true)
 |-- RatecodeID: string (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: string (nullable = true)
 |-- DOLocationID: string (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- fare_amount: string (nullable = true)
 |-- extra: string (nullable = true)
 |-- mta_tax: string (nullable = true)
 |-- tip_amount: string (nullable = true)
 |-- tolls_amount: string (nullable = true)
 |-- improvement_surcharge: string (nullable = true)
 |-- total_amount: string (nullable = true)
 |-- congestion_surcharge: string (nullable = true)
 |-- airport_fee: double (nullable = true)
 
 -parquet
 root
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)
'''

root
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)



'\n-csv\nroot\n |-- _c0: string (nullable = true)\n |-- VendorID: string (nullable = true)\n |-- tpep_pickup_datetime: string (nullable = true)\n |-- tpep_dropoff_datetime: string (nullable = true)\n |-- passenger_count: string (nullable = true)\n |-- trip_distance: string (nullable = true)\n |-- RatecodeID: string (nullable = true)\n |-- store_and_fwd_flag: string (nullable = true)\n |-- PULocationID: string (nullable = true)\n |-- DOLocationID: string (nullable = true)\n |-- payment_type: string (nullable = true)\n |-- fare_amount: string (nullable = true)\n |-- extra: string (nullable = true)\n |-- mta_tax: string (nullable = true)\n |-- tip_amount: string (nullable = true)\n |-- tolls_amount: string (nullable = true)\n |-- improvement_surcharge: string (nullable = true)\n |-- total_amount: string (nullable = true)\n |-- congestion_surcharge: string (nullable = true)\n |-- airport_fee: double (nullable = true)\n \n -parquet\n root\n |-- VendorID: long (nullable = true)\n |-- tpep_pi

In [52]:
# from pyspark.sql.functions import col

# # trips_df = trips_df.withColumn("trip_distance", col("trip_distance").cast("double"))
# # trips_df = trips_df.withColumn("total_amount", col("total_amount").cast("double"))
# # trips_df = trips_df.withColumn("passenger_count", col("passenger_count").cast("integer"))
# trips_df = trips_df.withColumn("total_amount", col("total_amount").cast("double")).withColumn("VendorID", col("VendorID").cast("integer")).withColumn("tpep_pickup_datetime", col("tpep_pickup_datetime").cast("string")).withColumn("tpep_dropoff_datetime", col("tpep_dropoff_datetime").cast("string")).withColumn("passenger_count", col("passenger_count").cast("integer")).withColumn("trip_distance", col("trip_distance").cast("double")).withColumn("RatecodeID", col("RatecodeID").cast("integer")).withColumn("store_and_fwd_flag", col("store_and_fwd_flag").cast("string")).withColumn("PULocationID", col("PULocationID").cast("integer")).withColumn("DOLocationID", col("DOLocationID").cast("integer")).withColumn("payment_type", col("payment_type").cast("integer")).withColumn("fare_amount", col("fare_amount").cast("double")).withColumn("extra", col("extra").cast("double")).withColumn("mta_tax", col("mta_tax").cast("double")).withColumn("tip_amount", col("tip_amount").cast("double")).withColumn("tolls_amount", col("tolls_amount").cast("double")).withColumn("improvement_surcharge", col("improvement_surcharge").cast("double")).withColumn("congestion_surcharge", col("congestion_surcharge").cast("double"))

# trips_df.printSchema()

# '''
# root
#  |-- VendorID: integer (nullable = true)
#  |-- tpep_pickup_datetime: string (nullable = true)
#  |-- tpep_dropoff_datetime: string (nullable = true)
#  |-- passenger_count: integer (nullable = true)
#  |-- trip_distance: double (nullable = true)
#  |-- RatecodeID: integer (nullable = true)
#  |-- store_and_fwd_flag: string (nullable = true)
#  |-- PULocationID: integer (nullable = true)
#  |-- DOLocationID: integer (nullable = true)
#  |-- payment_type: integer (nullable = true)
#  |-- fare_amount: double (nullable = true)
#  |-- extra: double (nullable = true)
#  |-- mta_tax: double (nullable = true)
#  |-- tip_amount: double (nullable = true)
#  |-- tolls_amount: double (nullable = true)
#  |-- improvement_surcharge: double (nullable = true)
#  |-- total_amount: double (nullable = true)
#  |-- congestion_surcharge: double (nullable = true)
# '''

In [6]:
# SQL을 사용하기 위해 TempView에 담는다.
trips_df.createOrReplaceTempView("trips")


# 분석 때 진행했던 이상치 제거를 적용한다.
query = """
SELECT 
    trip_distance,
    total_amount
FROM
    trips
WHERE
    total_amount < 5000
    AND total_amount > 0
    AND trip_distance > 0
    AND trip_distance < 500
    AND passenger_count < 5
    AND TO_DATE(tpep_pickup_datetime) >= '2021-01-01'
    AND TO_DATE(tpep_pickup_datetime) < '2021-08-01'
"""


# 쿼리를 적용시킨 데이터를 만들어 TempView에 담는다.
data_df = spark.sql(query)
data_df.createOrReplaceTempView("data")


data_df.show()

'''
+-------------+------------+
|trip_distance|total_amount|
+-------------+------------+
|          2.1|        11.8|
|          0.2|         4.3|
|         14.7|       51.95|
|         10.6|       36.35|
|         4.94|       24.36|
|          1.6|       14.15|
|          4.1|        17.3|
|          5.7|        21.8|
|          9.1|        28.8|
|          2.7|       18.95|
|         6.11|        24.3|
|         1.21|       10.79|
|          7.4|       33.92|
|         1.01|        10.3|
|         0.73|       12.09|
|         1.17|       12.36|
|         0.78|        9.96|
|         1.66|        12.3|
|         0.93|         9.3|
|         1.16|       11.84|
+-------------+------------+

+-------------+------------+
|trip_distance|total_amount|
+-------------+------------+
|         16.5|       70.07|
|         1.13|       11.16|
|         2.68|       18.59|
|         12.4|        43.8|
|          9.7|        32.3|
|          9.3|       43.67|
|         9.58|        46.1|
|         16.2|        45.3|
|         3.58|        19.3|
|         0.91|        14.8|
|         2.57|        12.8|
|          0.4|         5.3|
|         3.26|        17.3|
|        13.41|       47.25|
|         18.3|       61.42|
|         1.53|       14.16|
|          2.0|        11.8|
|         16.6|       54.96|
|         15.5|       56.25|
|          1.3|        16.8|
+-------------+------------+
'''

+-------------+------------+
|trip_distance|total_amount|
+-------------+------------+
|         16.5|       70.07|
|         1.13|       11.16|
|         2.68|       18.59|
|         12.4|        43.8|
|          9.7|        32.3|
|          9.3|       43.67|
|         9.58|        46.1|
|         16.2|        45.3|
|         3.58|        19.3|
|         0.91|        14.8|
|         2.57|        12.8|
|          0.4|         5.3|
|         3.26|        17.3|
|        13.41|       47.25|
|         18.3|       61.42|
|         1.53|       14.16|
|          2.0|        11.8|
|         16.6|       54.96|
|         15.5|       56.25|
|          1.3|        16.8|
+-------------+------------+
only showing top 20 rows



'\n+-------------+------------+\n|trip_distance|total_amount|\n+-------------+------------+\n|          2.1|        11.8|\n|          0.2|         4.3|\n|         14.7|       51.95|\n|         10.6|       36.35|\n|         4.94|       24.36|\n|          1.6|       14.15|\n|          4.1|        17.3|\n|          5.7|        21.8|\n|          9.1|        28.8|\n|          2.7|       18.95|\n|         6.11|        24.3|\n|         1.21|       10.79|\n|          7.4|       33.92|\n|         1.01|        10.3|\n|         0.73|       12.09|\n|         1.17|       12.36|\n|         0.78|        9.96|\n|         1.66|        12.3|\n|         0.93|         9.3|\n|         1.16|       11.84|\n+-------------+------------+\n\n+-------------+------------+\n|trip_distance|total_amount|\n+-------------+------------+\n|         16.5|       70.07|\n|         1.13|       11.16|\n|         2.68|       18.59|\n|         12.4|        43.8|\n|          9.7|        32.3|\n|          9.3|       43.67|\n|    

In [54]:
# 통계 확인
data_df.describe().show()
'''
+-------+------------------+-----------------+
|summary|     trip_distance|     total_amount|
+-------+------------------+-----------------+
|  count|          11804201|         11804201|
|   mean|1.1729952353403632|3.909488148346151|
| stddev|1.3025219630222131|5.617459072975492|
|    min|              0.01|             0.01|
|    max|             427.7|           2292.4|
+-------+------------------+-----------------+

+-------+------------------+------------------+
|summary|     trip_distance|      total_amount|
+-------+------------------+------------------+
|  count|          10797478|          10797478|
|   mean|2.8042136052510713|17.672030619622447|
| stddev|3.6919267970694842|12.548547997310054|
|    min|              0.01|              0.01|
|    max|             475.5|            4973.3|
+-------+------------------+------------------+
'''

+-------+------------------+------------------+
|summary|     trip_distance|      total_amount|
+-------+------------------+------------------+
|  count|          10797478|          10797478|
|   mean|2.8042136052510713|17.672030619622447|
| stddev|3.6919267970694842|12.548547997310054|
|    min|              0.01|              0.01|
|    max|             475.5|            4973.3|
+-------+------------------+------------------+



'\n+-------+------------------+-----------------+\n|summary|     trip_distance|     total_amount|\n+-------+------------------+-----------------+\n|  count|          11804201|         11804201|\n|   mean|1.1729952353403632|3.909488148346151|\n| stddev|1.3025219630222131|5.617459072975492|\n|    min|              0.01|             0.01|\n|    max|             427.7|           2292.4|\n+-------+------------------+-----------------+\n\n+-------+------------------+------------------+\n|summary|     trip_distance|      total_amount|\n+-------+------------------+------------------+\n|  count|          10797478|          10797478|\n|   mean|2.8042136052510713|17.672030619622447|\n| stddev|3.6919267970694842|12.548547997310054|\n|    min|              0.01|              0.01|\n|    max|             475.5|            4973.3|\n+-------+------------------+------------------+\n'

In [7]:
# 트레인 테스트 스플릿
train_df, test_df = data_df.randomSplit([0.8, 0.2], seed=6)

print(train_df.count(), test_df.count())
'''
8637580 2159898
'''

8637580 2159898


'\n8637580 2159898\n'

In [8]:
# 벡터 어셈블러로 트레인이 가능한 상태로 변환한다.
from pyspark.ml.feature import VectorAssembler

vassembler = VectorAssembler(inputCols=["trip_distance"], outputCol="features")

vtrain_df = vassembler.transform(train_df)

vtrain_df.show()
'''
+-------------+------------+--------+
|trip_distance|total_amount|features|
+-------------+------------+--------+
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
+-------------+------------+--------+

+-------------+------------+--------+
|trip_distance|total_amount|features|
+-------------+------------+--------+
|         0.01|        3.05|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
+-------------+------------+--------+
'''



+-------------+------------+--------+
|trip_distance|total_amount|features|
+-------------+------------+--------+
|         0.01|        3.05|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
+-------------+------------+--------+
only showing top 20 rows



'\n+-------------+------------+--------+\n|trip_distance|total_amount|features|\n+-------------+------------+--------+\n|         0.01|         3.3|  [0.01]|\n|         0.01|         3.3|  [0.01]|\n|         0.01|         3.3|  [0.01]|\n|         0.01|         3.3|  [0.01]|\n|         0.01|         3.3|  [0.01]|\n|         0.01|         3.3|  [0.01]|\n|         0.01|         3.3|  [0.01]|\n|         0.01|         3.3|  [0.01]|\n|         0.01|         3.3|  [0.01]|\n|         0.01|         3.3|  [0.01]|\n|         0.01|         3.3|  [0.01]|\n|         0.01|         3.3|  [0.01]|\n|         0.01|         3.3|  [0.01]|\n|         0.01|         3.3|  [0.01]|\n|         0.01|         3.3|  [0.01]|\n|         0.01|         3.3|  [0.01]|\n|         0.01|         3.3|  [0.01]|\n|         0.01|         3.3|  [0.01]|\n|         0.01|         3.3|  [0.01]|\n|         0.01|         3.3|  [0.01]|\n+-------------+------------+--------+\n\n+-------------+------------+--------+\n|trip_distance|total

In [10]:
# 선형 회귀 모델 생성
from pyspark.ml.regression import LinearRegression

lr = LinearRegression(
    maxIter=50,
    labelCol="total_amount",
    featuresCol="features"
)


model = lr.fit(vtrain_df)
vtest_df = vassembler.transform(test_df)

# 예측
prediction = model.transform(vtest_df)

prediction.show()
'''
+-------------+------------+--------+--------------------+
|trip_distance|total_amount|features|          prediction|
+-------------+------------+--------+--------------------+
|         0.01|         3.3|  [0.01]|-0.10767058263533147|
|         0.01|         3.3|  [0.01]|-0.10767058263533147|
|         0.01|         3.3|  [0.01]|-0.10767058263533147|
|         0.01|         3.3|  [0.01]|-0.10767058263533147|
|         0.01|         3.3|  [0.01]|-0.10767058263533147|
|         0.01|         3.3|  [0.01]|-0.10767058263533147|
|         0.01|         3.3|  [0.01]|-0.10767058263533147|
|         0.01|         3.3|  [0.01]|-0.10767058263533147|
|         0.01|         3.3|  [0.01]|-0.10767058263533147|
|         0.01|         3.3|  [0.01]|-0.10767058263533147|
|         0.01|         3.3|  [0.01]|-0.10767058263533147|
|         0.01|         3.3|  [0.01]|-0.10767058263533147|
|         0.01|         3.3|  [0.01]|-0.10767058263533147|
|         0.01|         3.3|  [0.01]|-0.10767058263533147|
|         0.01|         3.8|  [0.01]|-0.10767058263533147|
|         0.01|         3.8|  [0.01]|-0.10767058263533147|
|         0.01|         3.8|  [0.01]|-0.10767058263533147|
|         0.01|         3.8|  [0.01]|-0.10767058263533147|
|         0.01|         3.8|  [0.01]|-0.10767058263533147|
|         0.01|         3.8|  [0.01]|-0.10767058263533147|
+-------------+------------+--------+--------------------+
-parquet
+-------------+------------+--------+-----------------+
|trip_distance|total_amount|features|       prediction|
+-------------+------------+--------+-----------------+
|         0.01|         3.3|  [0.01]|9.314504128440735|
|         0.01|         3.3|  [0.01]|9.314504128440735|
|         0.01|         3.3|  [0.01]|9.314504128440735|
|         0.01|         3.3|  [0.01]|9.314504128440735|
|         0.01|         3.3|  [0.01]|9.314504128440735|
|         0.01|         3.3|  [0.01]|9.314504128440735|
|         0.01|         3.3|  [0.01]|9.314504128440735|
|         0.01|         3.3|  [0.01]|9.314504128440735|
|         0.01|         3.3|  [0.01]|9.314504128440735|
|         0.01|         3.3|  [0.01]|9.314504128440735|
|         0.01|         3.3|  [0.01]|9.314504128440735|
|         0.01|         3.3|  [0.01]|9.314504128440735|
|         0.01|         3.3|  [0.01]|9.314504128440735|
|         0.01|         3.3|  [0.01]|9.314504128440735|
|         0.01|         3.3|  [0.01]|9.314504128440735|
|         0.01|         3.3|  [0.01]|9.314504128440735|
|         0.01|         3.3|  [0.01]|9.314504128440735|
|         0.01|         3.3|  [0.01]|9.314504128440735|
|         0.01|         3.3|  [0.01]|9.314504128440735|
|         0.01|         3.3|  [0.01]|9.314504128440735|
+-------------+------------+--------+-----------------+
'''

23/05/01 12:15:09 WARN Instrumentation: [df3406a0] regParam is zero, which might cause numerical instability and overfitting.


23/05/01 12:15:13 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/05/01 12:15:13 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS


23/05/01 12:15:20 WARN InstanceBuilder$NativeLAPACK: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


+-------------+------------+--------+-----------------+
|trip_distance|total_amount|features|       prediction|
+-------------+------------+--------+-----------------+
|         0.01|         3.3|  [0.01]|9.314504128440735|
|         0.01|         3.3|  [0.01]|9.314504128440735|
|         0.01|         3.3|  [0.01]|9.314504128440735|
|         0.01|         3.3|  [0.01]|9.314504128440735|
|         0.01|         3.3|  [0.01]|9.314504128440735|
|         0.01|         3.3|  [0.01]|9.314504128440735|
|         0.01|         3.3|  [0.01]|9.314504128440735|
|         0.01|         3.3|  [0.01]|9.314504128440735|
|         0.01|         3.3|  [0.01]|9.314504128440735|
|         0.01|         3.3|  [0.01]|9.314504128440735|
|         0.01|         3.3|  [0.01]|9.314504128440735|
|         0.01|         3.3|  [0.01]|9.314504128440735|
|         0.01|         3.3|  [0.01]|9.314504128440735|
|         0.01|         3.3|  [0.01]|9.314504128440735|
|         0.01|         3.3|  [0.01]|9.314504128

'\n+-------------+------------+--------+--------------------+\n|trip_distance|total_amount|features|          prediction|\n+-------------+------------+--------+--------------------+\n|         0.01|         3.3|  [0.01]|-0.10767058263533147|\n|         0.01|         3.3|  [0.01]|-0.10767058263533147|\n|         0.01|         3.3|  [0.01]|-0.10767058263533147|\n|         0.01|         3.3|  [0.01]|-0.10767058263533147|\n|         0.01|         3.3|  [0.01]|-0.10767058263533147|\n|         0.01|         3.3|  [0.01]|-0.10767058263533147|\n|         0.01|         3.3|  [0.01]|-0.10767058263533147|\n|         0.01|         3.3|  [0.01]|-0.10767058263533147|\n|         0.01|         3.3|  [0.01]|-0.10767058263533147|\n|         0.01|         3.3|  [0.01]|-0.10767058263533147|\n|         0.01|         3.3|  [0.01]|-0.10767058263533147|\n|         0.01|         3.3|  [0.01]|-0.10767058263533147|\n|         0.01|         3.3|  [0.01]|-0.10767058263533147|\n|         0.01|         3.3|  [0.01]|

In [ ]:
# 평가
display(model.summary.rootMeanSquaredError)
'''
4.162209376107027
-parquet
6.049206919041973

'''


display(model.summary.r2)
'''
0.5369832746976624
-parquet
0.7685399244705984

'''

# 해당 모델의 root mean squared error(RMSE)는 약 4.162이고, 
# R-squared(R²)는 약 0.537. RMSE는 모델이 예측한 값과 실제 값 사이의 제곱 오차를 평균한 값의 제곱근으로, 
# 작을수록 모델의 예측 성능이 더 우수. 

# R²는 모델이 데이터를 얼마나 잘 설명하는지를 나타내는 지표로, 
# 1에 가까울수록 모델이 데이터를 잘 설명한다는 것을 의미합니다. 
# 따라서 RMSE가 작고 R²가 크다면 해당 모델의 예측 성능이 좋다고 볼 수 있습니다.

6.049206919041973

0.7685399244705984

'\n0.5369832746976624\n\n0.7685399244705984\n\n'

In [11]:

# 실서비스에서 사용하기 위한 테스크
from pyspark.sql.types import DoubleType
distance_list = [1.1, 5.5, 10.5, 30.0]
distance_df = spark.createDataFrame(distance_list, DoubleType()).toDF("trip_distance")


distance_df.show()
'''
+-------------+
|trip_distance|
+-------------+
|          1.1|
|          5.5|
|         10.5|
|         30.0|
+-------------+
'''


vdistance_df = vassembler.transform(distance_df)


model.transform(vdistance_df).show()
'''
+-------------+--------+------------------+
|trip_distance|features|        prediction|
+-------------+--------+------------------+
|          1.1|   [1.1]|12.575020400219376|
|          5.5|   [5.5]| 25.73673746061022|
|         10.5|  [10.5]| 40.69323412014527|
|         30.0|  [30.0]| 99.02357109233196|
+-------------+--------+------------------+
'''

+-------------+
|trip_distance|
+-------------+
|          1.1|
|          5.5|
|         10.5|
|         30.0|
+-------------+

+-------------+--------+------------------+
|trip_distance|features|        prediction|
+-------------+--------+------------------+
|          1.1|   [1.1]|12.575020400219376|
|          5.5|   [5.5]| 25.73673746061022|
|         10.5|  [10.5]| 40.69323412014527|
|         30.0|  [30.0]| 99.02357109233196|
+-------------+--------+------------------+



'\n+-------------+--------+------------------+\n|trip_distance|features|        prediction|\n+-------------+--------+------------------+\n|          1.1|   [1.1]|12.575020400219376|\n|          5.5|   [5.5]| 25.73673746061022|\n|         10.5|  [10.5]| 40.69323412014527|\n|         30.0|  [30.0]| 99.02357109233196|\n+-------------+--------+------------------+\n'

In [12]:
# 예측에 쓰일 컬럼을 추가하고 전처리

query = """
SELECT 
    passenger_count,
    PULocationID as pickup_location_id,
    DOLocationID as dropoff_location_id,
    trip_distance,
    HOUR(tpep_pickup_datetime) as pickup_time,
    DATE_FORMAT(TO_DATE(tpep_pickup_datetime), 'EEEE') AS day_of_week,
    total_amount
FROM
    trips
WHERE
    total_amount < 5000
    AND total_amount > 0
    AND trip_distance > 0
    AND trip_distance < 500
    AND passenger_count < 5
    AND TO_DATE(tpep_pickup_datetime) >= '2021-01-01'
    AND TO_DATE(tpep_pickup_datetime) < '2021-08-01'
"""
data_df = spark.sql(query)
data_df.createOrReplaceTempView("data")


data_df.printSchema()
'''
root
 |-- passenger_count: double (nullable = true)
 |-- pickup_location_id: long (nullable = true)
 |-- dropoff_location_id: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- pickup_time: integer (nullable = true)
 |-- day_of_week: string (nullable = true)
 |-- total_amount: double (nullable = true)
'''


train_df, test_df = data_df.randomSplit([0.8, 0.2], seed=6)


# 나뉘어진 데이터 프레임을 저장해둔다.
data_dir = "./spark"


# 파퀘이 형태로 저장
# 압축률이 좋고 디스크 io가 적다
# 컬럼 기반 포맷이다.
train_df.write.format("parquet").mode("overwrite").save(f"{data_dir}/train/")
test_df.write.format("parquet").mode("overwrite").save(f"{data_dir}/test/")


# 파퀘이 형태로 불러오기
train_df = spark.read.parquet(f"{data_dir}/train/")
test_df = spark.read.parquet(f"{data_dir}/test/")

root
 |-- passenger_count: double (nullable = true)
 |-- pickup_location_id: long (nullable = true)
 |-- dropoff_location_id: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- pickup_time: integer (nullable = true)
 |-- day_of_week: string (nullable = true)
 |-- total_amount: double (nullable = true)



In [14]:
# 스테이지 파이프 라인
# 프리프로세싱을 위한 파이프 라인을 생성하여 전처리한다.
# 스트링 값을 숫자값으로 바꾸어 원핫인코딩을 진행한다.

from pyspark.ml.feature import OneHotEncoder, StringIndexer

# 카테고리 피쳐 설정
cat_feats = [
    "pickup_location_id",
    "dropoff_location_id",
    "day_of_week"
]

# 스테이지를 담는 배열
stages = []


for c in cat_feats:
    cat_indexer = StringIndexer(inputCol=c, outputCol= c + "_idx").setHandleInvalid("keep")
    onehot_encoder = OneHotEncoder(inputCols=[cat_indexer.getOutputCol()], outputCols=[c + "_onehot"])
    stages += [cat_indexer, onehot_encoder]

display(stages)
'''
[StringIndexer_d41a34ec6edf,
 OneHotEncoder_abf8d2b7e964,
 StringIndexer_9bddff377670,
 OneHotEncoder_c9b9419c1535,
 StringIndexer_6cb24fd11494,
 OneHotEncoder_f33a79dbe1ff]
'''

[StringIndexer_d41a34ec6edf,
 OneHotEncoder_abf8d2b7e964,
 StringIndexer_9bddff377670,
 OneHotEncoder_c9b9419c1535,
 StringIndexer_6cb24fd11494,
 OneHotEncoder_f33a79dbe1ff]

'\n[StringIndexer_ff1b1c4b32df,\n OneHotEncoder_4ea9b3272ed6,\n StringIndexer_65b49d3966df,\n OneHotEncoder_2e03a29c5802,\n StringIndexer_b6267b01a6ed,\n OneHotEncoder_5de1f0401482]\n'

In [15]:
# 숫자형 전처리: 벡터 어셈블러, 스탠다드 스칼라
from pyspark.ml.feature import VectorAssembler, StandardScaler

num_feats = [
    "passenger_count",
    "trip_distance",
    "pickup_time"
]

for n in num_feats:
    num_assembler = VectorAssembler(inputCols=[n], outputCol= n + "_vecotr")
    num_scaler = StandardScaler(inputCol=num_assembler.getOutputCol(), outputCol= n + "_scaled")
    stages += [num_assembler, num_scaler]

display(stages)
'''
[StringIndexer_ff1b1c4b32df,
 OneHotEncoder_4ea9b3272ed6,
 StringIndexer_65b49d3966df,
 OneHotEncoder_2e03a29c5802,
 StringIndexer_b6267b01a6ed,
 OneHotEncoder_5de1f0401482,
 VectorAssembler_a56b59cc4ac2,
 StandardScaler_898a128ab6df,
 VectorAssembler_ab4e638f04bc,
 StandardScaler_73b16bd04318,
 VectorAssembler_7063855370da,
 StandardScaler_c4e3960fc997]
'''


# 두가지를 프리프로세싱을 했는데 합치는 과정을 진행한다.
# 벡터 어셈블러를 이용하여 가능하다.
assembler_inputs = [c + "_onehot" for c in cat_feats] + [n + "_scaled" for n in num_feats]
assembler = VectorAssembler(inputCols=assembler_inputs, outputCol="feature_vector")
stages += [assembler]

display(stages)
'''
[StringIndexer_ff1b1c4b32df,
 OneHotEncoder_4ea9b3272ed6,
 StringIndexer_65b49d3966df,
 OneHotEncoder_2e03a29c5802,
 StringIndexer_b6267b01a6ed,
 OneHotEncoder_5de1f0401482,
 VectorAssembler_a56b59cc4ac2,
 StandardScaler_898a128ab6df,
 VectorAssembler_ab4e638f04bc,
 StandardScaler_73b16bd04318,
 VectorAssembler_7063855370da,
 StandardScaler_c4e3960fc997,
 VectorAssembler_37481243c479]
'''


# 스테이지로 파이프 라인 생성
from pyspark.ml import Pipeline

transform_stages = stages
pipeline = Pipeline(stages=transform_stages)
fitted_transformer = pipeline.fit(train_df)


# 적용
vtrain_df = fitted_transformer.transform(train_df)

[StringIndexer_d41a34ec6edf,
 OneHotEncoder_abf8d2b7e964,
 StringIndexer_9bddff377670,
 OneHotEncoder_c9b9419c1535,
 StringIndexer_6cb24fd11494,
 OneHotEncoder_f33a79dbe1ff,
 VectorAssembler_c78f01b4b23b,
 StandardScaler_24cc8d5e3b7d,
 VectorAssembler_3fc1b9e98c1e,
 StandardScaler_b6b634203b5f,
 VectorAssembler_4ca61a0c9669,
 StandardScaler_3bbdc1b1e32e]

[StringIndexer_d41a34ec6edf,
 OneHotEncoder_abf8d2b7e964,
 StringIndexer_9bddff377670,
 OneHotEncoder_c9b9419c1535,
 StringIndexer_6cb24fd11494,
 OneHotEncoder_f33a79dbe1ff,
 VectorAssembler_c78f01b4b23b,
 StandardScaler_24cc8d5e3b7d,
 VectorAssembler_3fc1b9e98c1e,
 StandardScaler_b6b634203b5f,
 VectorAssembler_4ca61a0c9669,
 StandardScaler_3bbdc1b1e32e,
 VectorAssembler_585ea374ab60]

In [16]:
from pyspark.ml.regression import LinearRegression

lr = LinearRegression(
    maxIter=50,
    solver="normal",
    labelCol="total_amount",
    featuresCol="feature_vector"
)


vtrain_df.printSchema()
'''
root
 |-- passenger_count: double (nullable = true)
 |-- pickup_location_id: long (nullable = true)
 |-- dropoff_location_id: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- pickup_time: integer (nullable = true)
 |-- day_of_week: string (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- pickup_location_id_idx: double (nullable = false)
 |-- pickup_location_id_onehot: vector (nullable = true)
 |-- dropoff_location_id_idx: double (nullable = false)
 |-- dropoff_location_id_onehot: vector (nullable = true)
 |-- day_of_week_idx: double (nullable = false)
 |-- day_of_week_onehot: vector (nullable = true)
 |-- passenger_count_vecotr: vector (nullable = true)
 |-- passenger_count_scaled: vector (nullable = true)
 |-- trip_distance_vecotr: vector (nullable = true)
 |-- trip_distance_scaled: vector (nullable = true)
 |-- pickup_time_vecotr: vector (nullable = true)
 |-- pickup_time_scaled: vector (nullable = true)
 |-- feature_vector: vector (nullable = true)
'''

root
 |-- passenger_count: double (nullable = true)
 |-- pickup_location_id: long (nullable = true)
 |-- dropoff_location_id: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- pickup_time: integer (nullable = true)
 |-- day_of_week: string (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- pickup_location_id_idx: double (nullable = false)
 |-- pickup_location_id_onehot: vector (nullable = true)
 |-- dropoff_location_id_idx: double (nullable = false)
 |-- dropoff_location_id_onehot: vector (nullable = true)
 |-- day_of_week_idx: double (nullable = false)
 |-- day_of_week_onehot: vector (nullable = true)
 |-- passenger_count_vecotr: vector (nullable = true)
 |-- passenger_count_scaled: vector (nullable = true)
 |-- trip_distance_vecotr: vector (nullable = true)
 |-- trip_distance_scaled: vector (nullable = true)
 |-- pickup_time_vecotr: vector (nullable = true)
 |-- pickup_time_scaled: vector (nullable = true)
 |-- feature_vector: vector (nullable =

'\nroot\n |-- passenger_count: double (nullable = true)\n |-- pickup_location_id: long (nullable = true)\n |-- dropoff_location_id: long (nullable = true)\n |-- trip_distance: double (nullable = true)\n |-- pickup_time: integer (nullable = true)\n |-- day_of_week: string (nullable = true)\n |-- total_amount: double (nullable = true)\n |-- pickup_location_id_idx: double (nullable = false)\n |-- pickup_location_id_onehot: vector (nullable = true)\n |-- dropoff_location_id_idx: double (nullable = false)\n |-- dropoff_location_id_onehot: vector (nullable = true)\n |-- day_of_week_idx: double (nullable = false)\n |-- day_of_week_onehot: vector (nullable = true)\n |-- passenger_count_vecotr: vector (nullable = true)\n |-- passenger_count_scaled: vector (nullable = true)\n |-- trip_distance_vecotr: vector (nullable = true)\n |-- trip_distance_scaled: vector (nullable = true)\n |-- pickup_time_vecotr: vector (nullable = true)\n |-- pickup_time_scaled: vector (nullable = true)\n |-- feature_vec

In [17]:
model = lr.fit(vtrain_df)

23/05/01 12:19:50 WARN Instrumentation: [74c133ba] regParam is zero, which might cause numerical instability and overfitting.


23/05/01 12:20:13 WARN Instrumentation: [74c133ba] Cholesky solver failed due to singular covariance matrix. Retrying with Quasi-Newton solver.
23/05/01 12:20:14 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
23/05/01 12:20:14 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


In [18]:
vtest_df = fitted_transformer.transform(test_df)


predictions = model.transform(vtest_df)


# 캐싱을하여 나중에 쓰기 쉽게 만든다.
predictions.cache()


predictions.select(["trip_distance", "day_of_week", "total_amount", "prediction"]).show()
'''
+-------------+-----------+------------+------------------+
|trip_distance|day_of_week|total_amount|        prediction|
+-------------+-----------+------------+------------------+
|          0.1|   Saturday|         6.3|10.966827276701556|
|          6.9|     Sunday|       32.25|26.309172606557226|
|          5.5|     Sunday|       26.75|23.774938082368692|
|          1.4|   Saturday|        16.0| 13.98917580098726|
|          3.1|   Saturday|       18.95|18.599299121660074|
|          4.7|     Sunday|       26.15|  22.1110709387005|
|          1.7|  Wednesday|        14.3|16.467713264256663|
|          1.1|    Tuesday|        11.8|14.621574899414863|
|          1.1|     Sunday|        12.3|13.577630566699884|
|          2.8|     Monday|        14.3|  16.5270134919678|
|          1.3|     Sunday|        11.8|13.201615722873873|
|          6.5|   Saturday|       32.75|26.989725231478467|
|          2.4|   Thursday|        16.6|17.623767039146898|
|          2.9|     Sunday|        15.3|18.569701165382632|
|          1.3|     Monday|       18.35|13.557798502397809|
|          1.8|     Sunday|       17.15|14.976661780113137|
|          6.7|   Saturday|        23.8| 26.45514273713569|
|          4.3|  Wednesday|        22.8|19.047145765224702|
|          4.2|     Sunday|        18.8|18.644536248651384|
|          1.2|     Monday|        13.5|11.780782711421425|
+-------------+-----------+------------+------------------+
'''

+-------------+-----------+------------+------------------+
|trip_distance|day_of_week|total_amount|        prediction|
+-------------+-----------+------------+------------------+
|          0.1|   Saturday|         6.3|10.966827276701556|
|          6.9|     Sunday|       32.25|26.309172606557226|
|          5.5|     Sunday|       26.75|23.774938082368692|
|          1.4|   Saturday|        16.0| 13.98917580098726|
|          3.1|   Saturday|       18.95|18.599299121660074|
|          4.7|     Sunday|       26.15|  22.1110709387005|
|          1.7|  Wednesday|        14.3|16.467713264256663|
|          1.1|    Tuesday|        11.8|14.621574899414863|
|          1.1|     Sunday|        12.3|13.577630566699884|
|          2.8|     Monday|        14.3|  16.5270134919678|
|          1.3|     Sunday|        11.8|13.201615722873873|
|          6.5|   Saturday|       32.75|26.989725231478467|
|          2.4|   Thursday|        16.6|17.623767039146898|
|          2.9|     Sunday|        15.3|

'\n+-------------+-----------+------------+------------------+\n|trip_distance|day_of_week|total_amount|        prediction|\n+-------------+-----------+------------+------------------+\n|         0.01|     Sunday|       22.25|  12.3875521857505|\n|          0.1|     Sunday|         3.3|10.744178027113051|\n|          6.1|     Monday|        26.3|28.652704157860498|\n|          7.1|     Monday|       35.76|31.174097556214498|\n|          0.9|     Sunday|         8.3|  7.66950512061783|\n|          2.7|     Sunday|        16.6|15.133976597996543|\n|          1.0|   Saturday|         7.3| 9.111650395437803|\n|          1.7|   Saturday|       14.75|12.508165587852046|\n|          0.6|     Friday|         5.8| 9.667477937462271|\n|          0.9|     Monday|        11.0|10.668972172418805|\n|          1.1|     Sunday|         9.3|11.048364111576014|\n|          0.2|     Sunday|         0.3| 7.720810167026536|\n|          2.8|     Sunday|        12.3|15.966585384151795|\n|          5.4|   Sat

In [19]:
display(model.summary.rootMeanSquaredError)
'''
5.738804270966456
'''


display(model.summary.r2)
'''
0.7920670195898503
'''

5.738804270966456

0.7920670195898503

'\n0.8922402466214665\n'

## 하이퍼 파라미터 튜닝

In [20]:
data_dir = "./spark"
train_df = spark.read.parquet(f"{data_dir}/train/")
test_df = spark.read.parquet(f"{data_dir}/test/")


# 하이퍼 파라미터에 쓸 데이터 프레임 생성
toy_df = train_df.sample(False, 0.1, seed=6)


from pyspark.ml.feature import OneHotEncoder, StringIndexer

cat_feats = [
    "pickup_location_id",
    "dropoff_location_id",
    "day_of_week"
]

stages = []

for c in cat_feats:
    cat_indexer = StringIndexer(inputCol=c, outputCol= c + "_idx").setHandleInvalid("keep")
    onehot_encoder = OneHotEncoder(inputCols=[cat_indexer.getOutputCol()], outputCols=[c + "_onehot"])
    stages += [cat_indexer, onehot_encoder]


from pyspark.ml.feature import VectorAssembler, StandardScaler

num_feats = [
    "passenger_count",
    "trip_distance",
    "pickup_time"
]

for n in num_feats:
    num_assembler = VectorAssembler(inputCols=[n], outputCol= n + "_vecotr")
    num_scaler = StandardScaler(inputCol=num_assembler.getOutputCol(), outputCol= n + "_scaled")
    stages += [num_assembler, num_scaler]


assembler_inputs = [c + "_onehot" for c in cat_feats] + [n + "_scaled" for n in num_feats]
assembler = VectorAssembler(inputCols=assembler_inputs, outputCol="feature_vector")
stages += [assembler]


from pyspark.ml import Pipeline
from pyspark.ml.regression import LinearRegression
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import RegressionEvaluator

lr = LinearRegression(
    maxIter=30,
    solver="normal",
    labelCol='total_amount',
    featuresCol='feature_vector'
)

# cv 파이프 라인 생성
cv_stages = stages + [lr]


cv_pipeline = Pipeline(stages=cv_stages)


param_grid = ParamGridBuilder()\
                .addGrid(lr.elasticNetParam, [0.1, 0.2, 0.3, 0.4, 0.5])\
                .addGrid(lr.regParam, [0.01, 0.02, 0.03, 0.04, 0.05])\
                .build()


cross_val = CrossValidator(estimator=cv_pipeline,
                           estimatorParamMaps=param_grid,
                           evaluator=RegressionEvaluator(labelCol="total_amount"),
                           numFolds=5)


cv_model = cross_val.fit(toy_df)


alpha = cv_model.bestModel.stages[-1]._java_obj.getElasticNetParam()
reg_param = cv_model.bestModel.stages[-1]._java_obj.getRegParam()

In [21]:
transform_stages = stages
pipeline = Pipeline(stages=transform_stages)
fitted_transformer = pipeline.fit(train_df)


vtrain_df = fitted_transformer.transform(train_df)



lr = LinearRegression(
    maxIter=50,
    solver="normal",
    labelCol="total_amount",
    featuresCol="feature_vector",
    elasticNetParam=alpha,
    regParam=reg_param,
)


model = lr.fit(vtrain_df)


vtest_df = fitted_transformer.transform(test_df)


predictions = model.transform(vtest_df)


predictions.cache()


predictions.select(["trip_distance", "day_of_week", "total_amount", "prediction"]).show()


display(model.summary.rootMeanSquaredError)
'''
5.7403807105106095
'''


display(model.summary.r2)
'''
0.7919527662438353
'''

# 모델이 단순해서 그런가 성능적 향상이 있지는 않았다.

+-------------+-----------+------------+------------------+
|trip_distance|day_of_week|total_amount|        prediction|
+-------------+-----------+------------+------------------+
|          0.1|   Saturday|         6.3|10.707370690237537|
|          6.9|     Sunday|       32.25|26.209966503595517|
|          5.5|     Sunday|       26.75|23.663692399804376|
|          1.4|   Saturday|        16.0|13.773067728702877|
|          3.1|   Saturday|       18.95|18.463543056635764|
|          4.7|     Sunday|       26.15|21.931520221865526|
|          1.7|  Wednesday|        14.3| 16.20363497182565|
|          1.1|    Tuesday|        11.8|14.382489594856782|
|          1.1|     Sunday|        12.3|13.350923689899655|
|          2.8|     Monday|        14.3| 16.41438986143074|
|          1.3|     Sunday|        11.8| 13.07866410317098|
|          6.5|   Saturday|       32.75| 27.02367007075635|
|          2.4|   Thursday|        16.6|17.312316342132807|
|          2.9|     Sunday|        15.3|

5.7403807105106095

0.7919527662438353

'\n0.8918408379788968\n'

In [24]:
model_dir = "./model"
model.save(model_dir)


from pyspark.ml.regression import LinearRegressionModel


lr_model = LinearRegressionModel().load(model_dir)


predictions = lr_model.transform(vtest_df)


predictions.show()

'''
+---------------+------------------+-------------------+-------------+-----------+-----------+------------+----------------------+-------------------------+-----------------------+--------------------------+---------------+------------------+----------------------+----------------------+--------------------+--------------------+------------------+--------------------+--------------------+------------------+
|passenger_count|pickup_location_id|dropoff_location_id|trip_distance|pickup_time|day_of_week|total_amount|pickup_location_id_idx|pickup_location_id_onehot|dropoff_location_id_idx|dropoff_location_id_onehot|day_of_week_idx|day_of_week_onehot|passenger_count_vecotr|passenger_count_scaled|trip_distance_vecotr|trip_distance_scaled|pickup_time_vecotr|  pickup_time_scaled|      feature_vector|        prediction|
+---------------+------------------+-------------------+-------------+-----------+-----------+------------+----------------------+-------------------------+-----------------------+--------------------------+---------------+------------------+----------------------+----------------------+--------------------+--------------------+------------------+--------------------+--------------------+------------------+
|            0.0|                 4|                  4|          0.1|          9|   Saturday|         6.3|                  62.0|         (262,[62],[1.0])|                   50.0|          (261,[50],[1.0])|            3.0|     (7,[3],[1.0])|                 [0.0]|                 [0.0]|               [0.1]|[0.02703456530719...|             [9.0]|[1.0603469694664798]|(533,[62,312,526,...|10.707370690237537|
|            0.0|                 4|                 74|          6.9|         20|     Sunday|       32.25|                  62.0|         (262,[62],[1.0])|                   34.0|          (261,[34],[1.0])|            5.0|     (7,[5],[1.0])|                 [0.0]|                 [0.0]|               [6.9]|[1.8653850061967225]|            [20.0]|[2.3563265988143995]|(533,[62,296,528,...|26.209966503595517|
|            0.0|                 4|                 75|          5.5|          0|     Sunday|       26.75|                  62.0|         (262,[62],[1.0])|                   19.0|          (261,[19],[1.0])|            5.0|     (7,[5],[1.0])|                 [0.0]|                 [0.0]|               [5.5]|[1.4869010918959382]|             [0.0]|               [0.0]|(533,[62,281,528,...|23.663692399804376|
|            0.0|                 4|                113|          1.4|         21|   Saturday|        16.0|                  62.0|         (262,[62],[1.0])|                   32.0|          (261,[32],[1.0])|            3.0|     (7,[3],[1.0])|                 [0.0]|                 [0.0]|               [1.4]|[0.37848391430078...|            [21.0]|[2.4741429287551195]|(533,[62,294,526,...|13.773067728702877|
|            0.0|                 4|                141|          3.1|          5|   Saturday|       18.95|                  62.0|         (262,[62],[1.0])|                    2.0|           (261,[2],[1.0])|            3.0|     (7,[3],[1.0])|                 [0.0]|                 [0.0]|               [3.1]|[0.8380715245231652]|             [5.0]|[0.5890816497035999]|(533,[62,264,526,...|18.463543056635764|
|            0.0|                 4|                143|          4.7|         11|     Sunday|       26.15|                  62.0|         (262,[62],[1.0])|                   21.0|          (261,[21],[1.0])|            5.0|     (7,[5],[1.0])|                 [0.0]|                 [0.0]|               [4.7]|[1.2706245694383473]|            [11.0]|[1.2959796293479198]|(533,[62,283,528,...|21.931520221865526|
|            0.0|                 4|                144|          1.7|          3|  Wednesday|        14.3|                  62.0|         (262,[62],[1.0])|                   42.0|          (261,[42],[1.0])|            1.0|     (7,[1],[1.0])|                 [0.0]|                 [0.0]|               [1.7]|[0.45958761022238...|             [3.0]|[0.3534489898221599]|(533,[62,304,524,...| 16.20363497182565|
|            0.0|                 4|                148|          1.1|          4|    Tuesday|        11.8|                  62.0|         (262,[62],[1.0])|                   41.0|          (261,[41],[1.0])|            4.0|     (7,[4],[1.0])|                 [0.0]|                 [0.0]|               [1.1]|[0.29738021837918...|             [4.0]|[0.4712653197628799]|(533,[62,303,527,...|14.382489594856782|
|            0.0|                 4|                148|          1.1|         17|     Sunday|        12.3|                  62.0|         (262,[62],[1.0])|                   41.0|          (261,[41],[1.0])|            5.0|     (7,[5],[1.0])|                 [0.0]|                 [0.0]|               [1.1]|[0.29738021837918...|            [17.0]|[2.0028776089922395]|(533,[62,303,528,...|13.350923689899655|
|            0.0|                 4|                161|          2.8|         19|     Monday|        14.3|                  62.0|         (262,[62],[1.0])|                    4.0|           (261,[4],[1.0])|            6.0|     (7,[6],[1.0])|                 [0.0]|                 [0.0]|               [2.8]|[0.7569678286015685]|            [19.0]|[2.2385102688736795]|(533,[62,266,529,...| 16.41438986143074|
|            0.0|                 4|                170|          1.3|          7|     Sunday|        11.8|                  62.0|         (262,[62],[1.0])|                    5.0|           (261,[5],[1.0])|            5.0|     (7,[5],[1.0])|                 [0.0]|                 [0.0]|               [1.3]|[0.35144934899358...|             [7.0]|[0.8247143095850398]|(533,[62,267,528,...| 13.07866410317098|
|            0.0|                 4|                179|          6.5|          8|   Saturday|       32.75|                  62.0|         (262,[62],[1.0])|                   82.0|          (261,[82],[1.0])|            3.0|     (7,[3],[1.0])|                 [0.0]|                 [0.0]|               [6.5]| [1.757246744967927]|             [8.0]|[0.9425306395257598]|(533,[62,344,526,...| 27.02367007075635|
|            0.0|                 4|                209|          2.4|         23|   Thursday|        16.6|                  62.0|         (262,[62],[1.0])|                   63.0|          (261,[63],[1.0])|            0.0|     (7,[0],[1.0])|                 [0.0]|                 [0.0]|               [2.4]| [0.648829567372773]|            [23.0]|[2.7097755886365595]|(533,[62,325,523,...|17.312316342132807|
|            0.0|                 4|                209|          2.9|         10|     Sunday|        15.3|                  62.0|         (262,[62],[1.0])|                   63.0|          (261,[63],[1.0])|            5.0|     (7,[5],[1.0])|                 [0.0]|                 [0.0]|               [2.9]|[0.7840023939087674]|            [10.0]|[1.1781632994071998]|(533,[62,325,528,...| 18.29332071864325|
|            0.0|                 4|                234|          1.3|          0|     Monday|       18.35|                  62.0|         (262,[62],[1.0])|                   11.0|          (261,[11],[1.0])|            6.0|     (7,[6],[1.0])|                 [0.0]|                 [0.0]|               [1.3]|[0.35144934899358...|             [0.0]|               [0.0]|(533,[62,273,529,...|13.435922003808804|
|            0.0|                 4|                249|          1.8|          4|     Sunday|       17.15|                  62.0|         (262,[62],[1.0])|                   23.0|          (261,[23],[1.0])|            5.0|     (7,[5],[1.0])|                 [0.0]|                 [0.0]|               [1.8]|[0.4866221755295798]|             [4.0]|[0.4712653197628799]|(533,[62,285,528,...|14.775682748278388|
|            0.0|                 7|                 93|          6.7|         22|   Saturday|        23.8|                  63.0|         (262,[63],[1.0])|                  189.0|         (261,[189],[1.0])|            3.0|     (7,[3],[1.0])|                 [0.0]|                 [0.0]|               [6.7]|[1.8113158755823249]|            [22.0]|[2.5919592586958395]|(533,[63,451,526,...| 26.24719572150431|
|            0.0|                 7|                107|          4.3|         22|  Wednesday|        22.8|                  63.0|         (262,[63],[1.0])|                   16.0|          (261,[16],[1.0])|            1.0|     (7,[1],[1.0])|                 [0.0]|                 [0.0]|               [4.3]|[1.1624863082095516]|            [22.0]|[2.5919592586958395]|(533,[63,278,524,...| 19.24808191026743|
|            0.0|                 7|                114|          4.2|         22|     Sunday|        18.8|                  63.0|         (262,[63],[1.0])|                   37.0|          (261,[37],[1.0])|            5.0|     (7,[5],[1.0])|                 [0.0]|                 [0.0]|               [4.2]| [1.135451742902353]|            [22.0]|[2.5919592586958395]|(533,[63,299,528,...| 18.77816754228869|
|            0.0|                 7|                145|          1.2|         20|     Monday|        13.5|                  63.0|         (262,[63],[1.0])|                   61.0|          (261,[61],[1.0])|            6.0|     (7,[6],[1.0])|                 [0.0]|                 [0.0]|               [1.2]|[0.3244147836863865]|            [20.0]|[2.3563265988143995]|(533,[63,323,529,...| 11.83749695917942|
+---------------+------------------+-------------------+-------------+-----------+-----------+------------+----------------------+-------------------------+-----------------------+--------------------------+---------------+------------------+----------------------+----------------------+--------------------+--------------------+------------------+--------------------+--------------------+------------------+
'''

+---------------+------------------+-------------------+-------------+-----------+-----------+------------+----------------------+-------------------------+-----------------------+--------------------------+---------------+------------------+----------------------+----------------------+--------------------+--------------------+------------------+--------------------+--------------------+------------------+
|passenger_count|pickup_location_id|dropoff_location_id|trip_distance|pickup_time|day_of_week|total_amount|pickup_location_id_idx|pickup_location_id_onehot|dropoff_location_id_idx|dropoff_location_id_onehot|day_of_week_idx|day_of_week_onehot|passenger_count_vecotr|passenger_count_scaled|trip_distance_vecotr|trip_distance_scaled|pickup_time_vecotr|  pickup_time_scaled|      feature_vector|        prediction|
+---------------+------------------+-------------------+-------------+-----------+-----------+------------+----------------------+-------------------------+----------------------